In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

/Users/admin/Projects/vocabulary_learning/notebooks
/Users/admin/Projects/vocabulary_learning


<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)

import datetime

<IPython.core.display.Javascript object>

# Overall

In [3]:
# Historical dataset
from src.data.get_dataset import get_historical_data
from src.data.make_historical_features import create_historical_features

historical_data = get_historical_data("data/raw/historical_data__feature.csv")
historical_data = create_historical_features(historical_data)

<IPython.core.display.Javascript object>

In [4]:
# Vocab dataset
from src.data.get_dataset import get_vocab
from src.data.make_vocab_features import create_vocab_features

vocab = get_vocab("data/raw/german_english__feature.csv")
vocab = create_vocab_features(vocab)

<IPython.core.display.Javascript object>

In [5]:
# Dataset
from src.data.make_dataset import merge_feature_datasets

dataset = merge_feature_datasets(historical_data, vocab)

<IPython.core.display.Javascript object>

In [6]:
# Vardict
from src.data.make_dataset import get_vardict

vardict = get_vardict()

<IPython.core.display.Javascript object>

In [7]:
# Transform datatype
from src.data.make_dataset import transform_type

dataset = transform_type(dataset, vardict)

<IPython.core.display.Javascript object>

In [8]:
[x for x in dataset.columns.tolist() if x not in vardict["all"]]

['id_vocab',
 'german_word',
 'english_word',
 'score_before',
 'score_before_other_language',
 'result',
 'guess',
 'question_time',
 'write_it_again',
 'datetime',
 'id_session',
 'day',
 'id_historical_data']

<IPython.core.display.Javascript object>

In [9]:
dataset[dataset["german_word"] == "stimmen"][
    ["german_word", "english_word"] + [vardict["target"]] + vardict["all"]
]

,german_word,english_word,result,nb_characters_german,nb_characters_english,nb_words_german,nb_words_english,levenshtein_distance_german_english,previous_score,previous_score_other_language,previous_levenshtein_distance_guess_answer,previous_question_time,previous_write_it_again_german,previous_write_it_again_english,past_occurrences_same_language,past_successes_same_language,past_fails_same_language,past_occurrences_any_language,past_successes_any_language,past_fails_any_language,week_number,day_week,hour,nb_words_session,days_since_last_occurrence_same_language,days_since_last_occurrence_any_language,days_since_last_success_same_language,days_since_last_success_any_language,days_since_first_occur_same_language,days_since_first_occur_any_language,previous_result,previous_correct_article,previous_only_missed_uppercase,previous_write_it_again_not_null,is_noun,is_verb,language_asked,previous_language_asked
0,stimmen,to vote,1.0,7.0,7.0,1.0,1.0,6.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,39.0,6.0,15.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0.0,1.0,german,None
1,stimmen,to vote,0.0,7.0,7.0,1.0,1.0,6.0,1.0,0.0,0.0,2.0,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,39.0,6.0,15.0,17.0,0.0,0.0,NaN,NaN,0.0,0.0,True,None,False,False,0.0,1.0,german,german
2,stimmen,to vote,1.0,7.0,7.0,1.0,1.0,6.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,2.0,1.0,1.0,39.0,6.0,15.0,36.0,NaN,0.0,NaN,NaN,NaN,NaN,None,None,None,None,0.0,1.0,english,None


<IPython.core.display.Javascript object>

# Historical dataset

In [ ]:
from src.data.get_dataset import get_historical_data

historical_data_test = get_historical_data("data/raw/historical_data__feature.csv")
historical_data_test

## Add features

In [ ]:
from src.data.make_historical_features import create_historical_features

historical_data_test = create_historical_features(historical_data_test)
historical_data_test

# Vocab dataset

In [ ]:
from src.data.get_dataset import get_vocab

vocab = get_vocab("data/raw/german_english__feature.csv")
vocab

## Add features

In [ ]:
from src.data.make_vocab_features import create_vocab_features

vocab_test = create_vocab_features(vocab)
vocab_test

# Mix datasets

In [ ]:
dataset = pd.merge(historical_data_test, vocab_test, on="id_vocab")
dataset.sort_values("datetime", inplace=True)
dataset

# Variable type transformation

In [ ]:
# List of feature columns

In [ ]:
dataset.columns.tolist()

In [ ]:
vardict = dict()

## Target

In [ ]:
vardict["target"] = "result"

In [ ]:
dataset[[vardict["target"]]]

## Numerical

In [ ]:
vardict["numerical"] = [
    "previous_occurrences_same_language",
    "previous_successes_same_language",
    "previous_fails_same_language",
    "previous_occurrences_any_language",
    "previous_successes_any_language",
    "previous_fails_any_language",
    "levenshtein_distance_guess_answer",
    "previous_question_time",
    "write_it_again_german",
    "write_it_again_english",
    "levenshtein_distance_german_english",
]

In [ ]:
for i_num_var in vardict["numerical"]:
    dataset[i_num_var] = dataset[i_num_var].astype(float)

In [ ]:
dataset[["german_word", "guess"] + vardict["numerical"]]

## Difference in time

In [ ]:
vardict["diff_time"] = [
    "days_since_last_occurrence_same_language",
    "days_since_last_occurrence_any_language",
    "days_since_last_success_same_language",
    "days_since_last_success_any_language",
    "days_since_first_occur_same_language",
    "days_since_first_occur_any_language",
]

In [ ]:
for i_diff_time_var in vardict["diff_time"]:
    dataset[i_diff_time_var] = dataset[i_diff_time_var].dt.days.astype(float)

In [ ]:
dataset[["german_word", "guess"] + vardict["diff_time"]]

## Boolean

In [ ]:
vardict["boolean"] = [
    "previous_result",
    "correct_article",
    "only_missed_uppercase",
    "write_it_again_not_null",
]

In [ ]:
for i_boolean_var in vardict["boolean"]:
    dataset.loc[~dataset[i_boolean_var].isna(), i_boolean_var] = dataset.loc[
        ~dataset[i_boolean_var].isna(), i_boolean_var
    ].astype("bool")

In [ ]:
dataset[["german_word", "guess"] + vardict["boolean"]]

## Categorical

In [ ]:
vardict["categorical"] = [
    "previous_language_asked",
]

In [ ]:
for i_categorical_var in vardict["categorical"]:
    dataset.loc[~dataset[i_categorical_var].isna(), i_categorical_var] = dataset.loc[
        ~dataset[i_categorical_var].isna(), i_categorical_var
    ].astype(str)

    dataset.loc[dataset[i_categorical_var].isna(), i_categorical_var] = None

In [ ]:
dataset[["german_word", "guess"] + vardict["categorical"]]